# Import the source data
The data is provided as a directory that is three levels deep (the third level is ommited in the following listing).
``` bash
fiete@ubu:~/Documents/studium/analyse_semi_und_unstrukturierter_daten$ tree -d -L 1 CAPTUM
CAPTUM
├── Allergic Diseases
├── ANA
├── Angioedema
├── anti-FcεRI
├── Antihistamine
├── Anti-IgE
├── anti-TPO IgE ratio
├── ASST
├── Basophil
├── BAT
├── BHRA
├── CRP
├── Cyclosporine
├── D-Dimer
├── dsDNA
├── Duration
├── Eosinophil
├── IL-24
├── Omalizumab
├── Severity
├── Thyroglobulin
├── Total IgE
└── TPO
```

To work further with the source data, it is useful to have a list of file paths for the pdfs. The following command creates a comma seraparated list of all pdf files in the `CAPTUM` source folder ([adapted from here](https://askubuntu.com/questions/311135/print-contents-of-a-directory-structure-filesystem-to-csv-on-the-command-line#311179)).  
`find CAPTUM -name '*.pdf' -printf '"CAPTUM/%P",\n' > files.csv`

In [62]:
from os import path, walk

path = './CAPTUM'

pdf_filepaths = []
for root, directories, files in os.walk(path, topdown=False):
	for name in files:
		pdf_filepaths.append(os.path.join(root, name))

pdf_filepaths[:5]

['./CAPTUM/CRP/ANA/Asero 2017.pdf',
 './CAPTUM/CRP/ANA/Magen 2015.pdf',
 './CAPTUM/CRP/Severity/Kolkhir 2017 .pdf',
 './CAPTUM/CRP/Severity/Baek 2014.pdf',
 './CAPTUM/CRP/Severity/Kasperska-Zajac 2015.pdf']

In [51]:
import pandas
pdf_filepaths = pandas.read_csv('files.csv', encoding= 'utf-8' ,header=None)
pdf_filepaths = pdf_filepaths[0]
pdf_filepaths.head()

0                   CAPTUM/CRP/ANA/Asero 2017.pdf
1                   CAPTUM/CRP/ANA/Magen 2015.pdf
2           CAPTUM/CRP/Severity/Kolkhir 2017 .pdf
3               CAPTUM/CRP/Severity/Baek 2014.pdf
4    CAPTUM/CRP/Severity/Kasperska-Zajac 2015.pdf
Name: 0, dtype: object

In [64]:
dir_name = './CAPTUM'
for root, dirs, files in os.walk(dir_name):
    for name in dirs + files:
        join_path = os.path.join(root, name)
        full_path = os.path.normpath(join_path)
        st = os.lstat(full_path)
        if stat.S_ISREG(st.st_mode):
            h = hashlib.sha224()
            with open(full_path) as f:
                h.update(f.read())
                hash = h.digest()
        else:
            hash = None
            output(full_path, st, hash)

NameError: name 'stat' is not defined

In [71]:
# https://stackoverflow.com/questions/16874598/how-do-i-calculate-the-md5-checksum-of-a-file-in-python#16876405
import hashlib
#hashlib.md5(pdf_filepaths.loc[0,0]).hexdigest()
#file_name = pdf_filepaths.loc[0] # 'CAPTUM/CRP/ANA/Asero 2017.pdf'
file_name = pdf_filepaths[0]

import sys
sys.stdout.encoding

import codecs

# Open,close, read file and calculate MD5 on its contents 
with open(file_name, 'rb') as file_to_check:
    # read contents of the file
    data = file_to_check.read()    
    # pipe contents of the file through
    md5_returned = hashlib.md5(data).hexdigest()

print(md5_returned)

2fad223ae2232cb9e855d3ece9e34b72
